In [18]:
import pandas as pd
df = pd.read_csv('../dataset/DBPEDIA_train.csv')
document_list = df['text'].to_numpy().astype('str')
document_list.dtype

dtype('<U3944')

In [19]:
# # Tokenizeing documents
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# encoded_list = []

# for i, doc in enumerate(document_list):
#     encoded = tokenizer.encode_plus(doc, add_special_tokens=True, max_length=512)
#     encoded_list.append(encoded)

In [20]:
# df['text'] = encoded_list

In [21]:
# encoding topics to create the adjacency matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np

labelEncoder = LabelEncoder()
topics = np.concatenate((df['l1'].unique(), df['l2'].unique(), df['l3'].unique()))
labelEncoder.fit(topics)

def encode_topic(topic):
    print(type(topic))
    return labelEncoder.transform(topic)

df['l1_encoded'] = labelEncoder.transform(df['l1'])
df['l2_encoded'] = labelEncoder.transform(df['l2'])
df['l3_encoded'] = labelEncoder.transform(df['l3'])


In [22]:
graph_dict = {}

for i, row in df.iterrows():
    l1 = row['l1_encoded']
    l2 = row['l2_encoded']
    l3 = row['l3_encoded']

    if l1 not in graph_dict:
        graph_dict[l1] = {}
    if l2 not in graph_dict[l1]:
        graph_dict[l1][l2] = {} 

    graph_dict[l1][l2][l3] = 1

In [23]:
# creating adjacency matrix
adj_matrix = np.zeros((len(topics), len(topics)))
square_numeric_dict = {'source': [], 'target': []}
for i, df_row in df.iterrows():
    l1 = df_row['l1_encoded']
    l2 = df_row['l2_encoded']
    l3 = df_row['l3_encoded']

    adj_matrix[l1, l2] = 1
    adj_matrix[l2, l3] = 1

for i, row in enumerate(adj_matrix):
    for j, value in enumerate(row):
        if value == 0:
            continue
        square_numeric_dict['source'].append(i)
        square_numeric_dict['target'].append(j)

square_numeric_edges = pd.DataFrame(square_numeric_dict)

In [24]:
# creating node features
x = np.arange(298)
x = labelEncoder.inverse_transform(x)
feature_array = x.reshape(298, -1)

In [25]:
# loading GloVe model to get topic word embeddings
# from https://stackoverflow.com/questions/37793118/load-pretrained-glove-vectors-in-python
import torch
import torchtext

glove = torchtext.vocab.GloVe(name="6B", dim=50)

In [47]:
# Creating "ego-graphs" (each node is seperated into a graph with itself, parent, and siblings)
# The base node (so the node itself) will be masked, aka. have a [MASK] embedding
# The sibling nodes need to have a negative relationship with the base node (so negative value in adjacency matrix?)
from spektral.data import Graph
import numpy as np
import re
def create_ego_graph(l1_topic, l2_topic, l3_topic, graph_dict):
    if l3_topic != None:
        siblings_list = list(graph_dict[l1_topic][l2_topic].keys())
        siblings_list.remove(l3_topic)
        base = l3_topic
        parent = l2_topic
        grandparent = l1_topic

        all_nodes_list = siblings_list.copy()
        all_nodes_list.append(base)
        all_nodes_list.append(parent)
        all_nodes_list.append(grandparent)
        
        n_nodes = len(all_nodes_list)
        adj_matrix = np.zeros((n_nodes, n_nodes))

        node_label_encoder = LabelEncoder()
        node_label_encoder.fit(all_nodes_list)
        
        encoded_base = node_label_encoder.transform([base])[0]
        encoded_parent = node_label_encoder.transform([parent])[0]
        encoded_grandparent = node_label_encoder.transform([grandparent])[0]

        adj_matrix[encoded_base][encoded_parent] = 1
        adj_matrix[encoded_parent][encoded_base] = 1

        adj_matrix[encoded_grandparent][encoded_parent] = 1
        adj_matrix[encoded_parent][encoded_grandparent] = 1

        for sibling in siblings_list: 
            encoded_sibling = node_label_encoder.transform([sibling])[0]
            adj_matrix[encoded_sibling][encoded_base] = -1
            adj_matrix[encoded_base][encoded_sibling] = -1
        
    elif l2_topic != None:
        siblings_list = list(graph_dict[l1_topic].keys())
        siblings_list.remove(l2_topic)
        base = l2_topic
        parent = l1_topic

        all_nodes_list = siblings_list.copy()
        all_nodes_list.append(base)
        all_nodes_list.append(parent)
        
        n_nodes = len(all_nodes_list)
        adj_matrix = np.zeros((n_nodes, n_nodes))

        node_label_encoder = LabelEncoder()
        node_label_encoder.fit(all_nodes_list)
        
        encoded_base = node_label_encoder.transform([base])[0]
        encoded_parent = node_label_encoder.transform([parent])[0]

        adj_matrix[encoded_base][encoded_parent] = 1
        adj_matrix[encoded_parent][encoded_base] = 1

        for sibling in siblings_list: 
            encoded_sibling = node_label_encoder.transform([sibling])[0]
            adj_matrix[encoded_sibling][encoded_base] = -1
            adj_matrix[encoded_base][encoded_sibling] = -1
    
    elif l1_topic != None:
        siblings_list = list(graph_dict.keys())
        siblings_list.remove(l1_topic)
        base = l1_topic

        all_nodes_list = siblings_list.copy()
        all_nodes_list.append(base)
        
        n_nodes = len(all_nodes_list)
        adj_matrix = np.zeros((n_nodes, n_nodes))

        node_label_encoder = LabelEncoder()
        node_label_encoder.fit(all_nodes_list)
        
        encoded_base = node_label_encoder.transform([base])[0]

        for sibling in siblings_list: 
            encoded_sibling = node_label_encoder.transform([sibling])[0]
            adj_matrix[encoded_sibling][encoded_base] = -1
            adj_matrix[encoded_base][encoded_sibling] = -1

    ego_features = np.zeros((n_nodes, 50))
    y = [0, 1] if l1_topic != 3 else [1, 0]
    encoded_nodes_list = node_label_encoder.transform(all_nodes_list)

    for i, node in enumerate(all_nodes_list):
        feature = feature_array[node]
        split_words_list = re.sub( r"([A-Z])", r" \1", feature[0]).split()
        n_words = len(split_words_list)
        embedding_avg = np.array([glove[word.lower()].numpy() for word in split_words_list]).sum(axis=0)/n_words
        
        # Masking base node, setting the embedding to all 0's
        if (node == base):
            embedding_avg = glove['MASK']

        ego_features[encoded_nodes_list[i]] = embedding_avg

    return Graph(a=adj_matrix, x=ego_features, y=y)

graph_list = []

for l1_topic in graph_dict:
    for l2_topic in graph_dict[l1_topic]:
        for l3_topic in graph_dict[l1_topic][l2_topic]:
            graph_list.append(create_ego_graph(l1_topic, l2_topic, l3_topic, graph_dict))
        graph_list.append(create_ego_graph(l1_topic, l2_topic, None, graph_dict))
    graph_list.append(create_ego_graph(l1_topic, None, None, graph_dict))

graph_list = np.array(graph_list)

In [48]:
from spektral.data import Dataset

class MyDataset(Dataset):
    """
    """
    def __init__(self, graph_list: list[Graph], **kwargs):
        self.graph_list = graph_list
        super().__init__(**kwargs)

    def read(self):
        # We must return a list of Graph objects
        num_l =  np.random.permutation(len(self.graph_list))
        return [self.graph_list[i] for i in num_l]
    
dataset = MyDataset(graph_list)

In [49]:
from spektral.data import DisjointLoader, BatchLoader

# Train/test split
np.random.shuffle(dataset)
split = int(0.6 * len(dataset))
data_tr, data_te = dataset[:split], dataset[split:]

loader_tr = DisjointLoader(data_tr, batch_size=3, epochs=50)
loader_te = DisjointLoader(data_te, batch_size=3,)

C:\Users\saif8\AppData\Local\Temp\ipykernel_36396\4187418381.py:4: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(dataset)


In [50]:
    ################################################################################
    # Build model
    ################################################################################
from tensorflow import int64
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from spektral.layers import GCNConv, GlobalAvgPool, GraphMasking

n_out = dataset.n_labels

    # class Net(Model):
    #     def __init__(self):
    #         super().__init__()
    #         self.conv1 = GCNConv(32, activation="relu")
    #         self.conv2 = GCNConv(32, activation="relu")
    #         self.global_pool = GlobalAvgPool()
    #         self.dense = Dense(n_out, activation='softmax')

    #     def call(self, inputs):
    #         x, a, i = inputs
    #         print(x, a, i)
    #         x = self.conv1([x, a])
    #         x = self.conv2([x, a])
    #         output = self.global_pool([x, i])
    #         output = self.dense(output)

    #         return output
        
X_in = Input(shape=(dataset.n_node_features))
A_in = Input(shape=(None,), sparse=True)
I_in = Input(shape=(), dtype=int64)

X = GCNConv(32, activation='relu')([X_in, A_in])
X = GCNConv(32, activation='relu')([X, A_in])
X = GlobalAvgPool()([X, I_in])
out = Dense(n_out, activation='softmax')(X)

model = Model(inputs=[X_in, A_in, I_in], outputs=out)


In [51]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
import tensorflow as tf
learning_rate = 1e-3  # Learning rate
epochs = 10  # Number of training epochs
batch_size = 32  # Batch size

# model = Net()
optimizer = Adam(learning_rate)
loss_fn = CategoricalCrossentropy()

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 gcn_conv_4 (GCNConv)           (None, 32)           1632        ['input_7[0][0]',                
                                                                  'input_8[0][0]']                
                                                                                                  
 gcn_conv_5 (GCNConv)           (None, 32)           1056        ['gcn_conv_4[0][0]',       

In [52]:
model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs)

Epoch 1/10
15/60 [======>.......................] - ETA: 0s - loss: 0.9262 - accuracy: 0.5778 

c:\Users\saif8\miniconda3\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


60/60 [==============================] - 2s 11ms/step - loss: 0.5719 - accuracy: 0.6910
Epoch 2/10
60/60 [==============================] - 1s 12ms/step - loss: 0.2828 - accuracy: 0.9551
Epoch 3/10
60/60 [==============================] - 1s 10ms/step - loss: 0.1922 - accuracy: 0.9831
Epoch 4/10
60/60 [==============================] - 1s 11ms/step - loss: 0.1301 - accuracy: 0.9888
Epoch 5/10
60/60 [==============================] - 1s 11ms/step - loss: 0.0894 - accuracy: 0.9944
Epoch 6/10
60/60 [==============================] - 1s 13ms/step - loss: 0.0683 - accuracy: 0.9944
Epoch 7/10
60/60 [==============================] - 1s 11ms/step - loss: 0.0521 - accuracy: 0.9944
Epoch 8/10
60/60 [==============================] - 1s 17ms/step - loss: 0.0449 - accuracy: 0.9944
Epoch 9/10
60/60 [==============================] - 1s 13ms/step - loss: 0.0347 - accuracy: 0.9944
Epoch 10/10
60/60 [==============================] - 1s 11ms/step - loss: 0.0334 - accuracy: 0.9944


In [53]:
################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done. Test loss: {}".format(loss))

Testing model
40/40 [==============================] - 1s 10ms/step - loss: 0.0275 - accuracy: 1.0000
Done. Test loss: [0.02752322517335415, 1.0]
